# COGS 108 - Final Project

## Important

- ONE, and only one, member of your group should upload this notebook to TritonED. 
- Each member of the group will receive the same grade on this assignment. 
- Keep the file name the same: submit the file 'FinalProject.ipynb'.
- Only upload the .ipynb file to TED, do not upload any associted data. Make sure that for cells in which you want graders to see output that these cells have been executed.

## Group Members: Fill in the Student IDs of each group member here

Replace the lines below to list each persons full student ID, ucsd email and full name.

-  James McDougall - A14947178
- 
- 
- 



# Part 0 - Importing the required packages

In [1]:
import pandas as pd # for use of dataframes
import numpy as np
import matplotlib.pyplot as plt # for plotting graphs 
import statsmodels.api as sm # for fitting an OLS model
import scipy.stats as stats # for access to tests for normal distribution
import statsmodels.formula.api as smf # for binomial distribution/logistic regression
from scipy.stats import ttest_ind, normaltest # for access to t tests, normal test

# Part 1 - Introduction and Background

Chicago, otherwise known as 'The Windy City', has some of the worst crime rates in America. Some of this has to do with statitical flukes; it is the third largest city in the U.S., which makes any type of criminal activity more easily reportable. However, most people wouldn't argue over the real danger you are in when you go to some of the worse off areas of Chicago.

We wanted to analyze the crime rates in Chicago in some way in order to assist people in the city in understanding what problems exist and where they are happening. In the end, we decided to ask the following question:

"Does the district a crime is committed in or type of crime more closely predict whether an arrest was made".

The answers derived from the answer to this question could spawn further analysis, e.g.:

1. If a particular crime is more likely to result in an arrest in one district over the other, why is that? Is it the police in that district? Is it the citizens? Or is it the geography?

2. If the type of crime affects whether an arrest is made, does that mean that we are more afraid of certain crimes over others? Is this a valid fear? 

# Part 2 - Data Description

We are using this dataset: https://www.kaggle.com/chicago/chicago-crime

This is chicago crime data from 2001 to present. It represents all reported instances of crime, and whether or not an arrest has been made since the data was last released. Each row represents a crime, except for multiple homicides, for which there is a different row for each victim of the crime. 

Here are some basic stats:

In [2]:
crime = pd.read_csv('~/data/CrimesSmall.csv')

In [3]:
crime.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,7956475,HT188097,3/4/2011 10:52,025XX S KILDARE AVE,554,ASSAULT,AGG PO HANDS NO/MIN INJURY,"SCHOOL, PUBLIC, BUILDING",True,False,...,22.0,30.0,08A,1148345.0,1886897.0,2011,2/10/2018 15:50,41.845586,-87.731082,"(41.84558589, -87.731082337)"
1,7956476,HT185132,3/4/2011 12:00,024XX W 45TH PL,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,RESIDENCE,False,False,...,12.0,58.0,11,1160803.0,1874437.0,2011,2/10/2018 15:50,41.811145,-87.685707,"(41.811145384, -87.685707211)"
2,7956477,HT187881,3/4/2011 9:00,023XX N CLYBOURN AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,...,32.0,7.0,14,1165097.0,1915872.0,2011,2/10/2018 15:50,41.924757,-87.668781,"(41.924756753, -87.668780552)"
3,7956479,HT187939,3/3/2011 8:00,003XX W 100TH ST,890,THEFT,FROM BUILDING,RESIDENTIAL YARD (FRONT/BACK),False,False,...,9.0,49.0,6,1175827.0,1838618.0,2011,2/10/2018 15:50,41.712530,-87.631672,"(41.712529985, -87.631672138)"
4,7956480,HT187986,3/4/2011 10:35,065XX W BRYN MAWR AVE,460,BATTERY,SIMPLE,"SCHOOL, PUBLIC, BUILDING",True,False,...,41.0,10.0,08B,1131583.0,1936817.0,2011,2/10/2018 15:50,41.982879,-87.791441,"(41.98287878, -87.791440835)"


In [4]:
crime.describe()

,ID,Beat,District,Ward,Community Area,X Coordinate,Y Coordinate,Year,Latitude,Longitude
count,1.999000e+03,1999.000000,1999.000000,1991.000000,1989.000000,1.988000e+03,1.988000e+03,1999.000000,1988.000000,1988.000000
mean,7.892279e+06,1199.812406,11.340170,22.699146,38.602313,1.164586e+06,1.883800e+06,2010.863432,41.836745,-87.671610
std,5.965758e+05,705.621225,6.976461,13.437493,21.603693,1.608752e+04,3.185389e+04,0.951716,0.087604,0.058523
min,1.961551e+06,111.000000,1.000000,1.000000,1.000000,1.117615e+06,1.814534e+06,2001.000000,41.646343,-87.842868
25%,7.957218e+06,621.000000,6.000000,11.000000,23.000000,1.152836e+06,1.857924e+06,2011.000000,41.765687,-87.714182
50%,7.957867e+06,1111.000000,10.000000,22.000000,34.000000,1.165511e+06,1.887310e+06,2011.000000,41.846341,-87.667811
75%,7.958476e+06,1732.000000,17.000000,34.000000,60.000000,1.176520e+06,1.908376e+06,2011.000000,41.904272,-87.627847
max,7.959109e+06,2535.000000,25.000000,50.000000,77.000000,1.203816e+06,1.951242e+06,2011.000000,42.021808,-87.529246


In [5]:
print('Total Number or Crimes:\t\t\t{0}'.format(len(crime)))
print('Total number of Unique Districts:\t{0}'.format(len(crime['District'].unique())))

Total Number or Crimes:			1999
Total number of Unique Districts:	22


Some crimes appear more than others. Later on, we might have to select the top crimes to analyze, so here they are

In [6]:
crime['Primary Type'].value_counts().nlargest(3)

BATTERY      418
THEFT        383
NARCOTICS    266
Name: Primary Type, dtype: int64

# Part 3 - Data Cleaning and Preprocessing

This data is fairly clean, because it is maintained by the Chicago Police Department, which has high organizational standards. There aren't very many invalid crimes or unknowns.

To clean the data, we will do the following (not necessarily in order):

1. Drop useless columns
2. Standardize the location description
3. Standardize the crime type to get rid of the uppercase, standardize the non-criminal type
4. Convert True to 1.0 and False to 0.0
5. Remove NaN values

In [7]:
# first, drop the location description column and community area column because 
crime = crime.drop(['Location Description', 'Block', 'Community Area','Latitude', 'Beat', 'Ward','Longitude','X Coordinate','Y Coordinate','Location','Ward'],axis=1)

In [8]:
"""
    Author: James McDougall
    Param: string - is the string which is the name of the Primary Type
    Returns: a variable of type str which is lower case and represents  a more standardized type
"""
def standardize_primary_type(string):
    # compile all non-criminal offenses into on label
    if string == 'NON-CRIMINAL (SUBJECT SPECIFIED)' or string == 'NON - CRIMINAL' or string == 'NON-CRIMINAL':
        return 'non-criminal'
    if string == 'OTHER OFFENSE':
        return 'other'
    # rename crim sexual assault to just sexual assault to make it easier to read
    if string == 'CRIM SEXUAL ASSAULT':
        return 'sexual assault'
    else:
        # everything else, make sure to lowercase it so we don't have to use caps lock lol
        return string.lower()
        

Removing NaNs. Check that the two important columns don't have NaNs.

In [9]:
to_drop = crime[ crime['District'].isnull()]
print('Number of rows to drop with NaN district: ' + str(len(to_drop)))
crime = crime.drop(to_drop.index,axis=0)

to_drop = crime[crime['Primary Type'].isnull()]
print('Number of row to drop with NaN Primary Type: ' + str(len(to_drop)))
crime = crime.drop(to_drop.index,axis=0)

Number of rows to drop with NaN district: 0
Number of row to drop with NaN Primary Type: 0


To standardize the data we do the following:
1. standardize the type to be all lower case, merge some similar crimes that are formatted differently together
2. convert district, which is a float, to a string, because district should be categorical, not numerical.

In [11]:
crime['Type'] = crime['Primary Type'].apply(standardize_primary_type)
crime['District'] = crime['District'].astype(str)

def arrest_to_int(string):
    if string == True:
        return 1
    elif string == False:
        return 0
crime['Arrest'] = crime['Arrest'].apply(arrest_to_int)

# Part 4 - Data Visualization

# Part 5 - Data Analysis and Results

We are trying to predict whether the type of crime ('Type') or the district ('District') is more predictive of whether an arrest was made.

In [12]:
#binomial_model = smf.glm(formula='ArrestInts ~ Type + District', data=crimes.loc[ind,:] , family=sm.families.Binomial())
binomial_model = smf.glm(formula='Arrest ~ Type + District', data=crime, family=sm.families.Binomial())
binomial_results = binomial_model.fit()
binomial_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 Arrest   No. Observations:                 1999
Model:                            GLM   Df Residuals:                     1955
Model Family:                Binomial   Df Model:                           43
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -683.47
Date:                Fri, 15 Mar 2019   Deviance:                       1366.9
Time:                        19:58:38   Pearson chi2:                 1.73e+03
No. Iterations:                    24   Covariance Type:             nonrobust
============================================================================================================
                                               coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                  -24.6008   1.07e+05     -0.000      1.000    -2.1e+05     2.1e+05
Type[T.assault]                             23.6977   1.07e+05      0.000      1.000    -2.1e+05     2.1e+05
Type[T.battery]                             24.3328   1.07e+05      0.000      1.000    -2.1e+05     2.1e+05
Type[T.burglary]                            21.7548   1.07e+05      0.000      1.000    -2.1e+05     2.1e+05
Type[T.criminal damage]                     23.0107   1.07e+05      0.000      1.000    -2.1e+05     2.1e+05
Type[T.criminal trespass]                   26.6990   1.07e+05      0.000      1.000    -2.1e+05     2.1e+05
Type[T.deceptive practice]                  24.0417   1.07e+05      0.000      1.000    -2.1e+05     2.1e+05
Type[T.gambling]                            50.9848   2.41e+05      0.000      1.000   -4.72e+05    4.73e+05
Type[T.interference with public officer]    50.7528   1.63e+05      0.000      1.000    -3.2e+05     3.2e+05
Type[T.intimidation]                         0.0330   2.41e+05   1.37e-07      1.000   -4.72e+05    4.72e+05
Type[T.kidnapping]                           0.0422   1.84e+05    2.3e-07      1.000    -3.6e+05     3.6e+05
Type[T.liquor law violation]                50.8116   1.64e+05      0.000      1.000   -3.22e+05    3.22e+05
Type[T.motor vehicle theft]                 22.3333   1.07e+05      0.000      1.000    -2.1e+05     2.1e+05
Type[T.narcotics]                           51.0373   1.08e+05      0.000      1.000   -2.11e+05    2.11e+05
Type[T.offense involving children]          24.3751   1.07e+05      0.000      1.000    -2.1e+05     2.1e+05
Type[T.other]                               23.7017   1.07e+05      0.000      1.000    -2.1e+05     2.1e+05
Type[T.prostitution]                        50.7389   1.38e+05      0.000      1.000   -2.71e+05    2.71e+05
Type[T.public peace violation]              26.5798   1.07e+05      0.000      1.000    -2.1e+05     2.1e+05
Type[T.robbery]                             21.2705   1.07e+05      0.000      1.000    -2.1e+05     2.1e+05
Type[T.sex offense]                         25.5296   1.07e+05      0.000      1.000    -2.1e+05     2.1e+05
Type[T.sexual assault]                      23.5640   1.07e+05      0.000      1.000    -2.1e+05     2.1e+05
Type[T.theft]                               23.4497   1.07e+05      0.000      1.000    -2.1e+05     2.1e+05
Type[T.weapons violation]                   26.9081   1.07e+05      0.000      1.000    -2.1e+05     2.1e+05
District[T.10]                              -1.1510      0.466     -2.471      0.013      -2.064      -0.238
District[T.11]                              -1.0760      0.443     -2.428      0.015      -1.945      -0.208
District[T.12]                              -0.9042      0.475     -1.905      0.057      -1.834       0.026
D

# Part 6 - Privacy/Ethics Considerations

The data is private because there is no identifying information. There are no names, only crime ids and description. The only identifying information is the location. This could be an issue because some of the crimes occurred in apartments, so you could conceivably find out the location where a crime was committed and use this to find someone.

A potential ethics considerations involving this data is that we are publicizing the locations of high crime, which could be used to discriminate against communities. While this data is publicily available, we are organizing and formatting it in a way that can be used to show which districts have a high crime rate. People with ill intentions could use this to argue that no one should go near these areas, alientating those communities.

# Part 7 - Conclusions and Discussion